# ENV

In [ ]:
# minVolumeTrade
# sleep_api

# TRANSFORMAR

In [ ]:
# updatePast_Asks_Bids() -> GCP
# history_trades() -> REVISAR QUOTATION
# balancing_Ask_Bid() -> ENVÍO DE ALERTA
# REVISAR ROUNDS

# LIBRARIES

In [ ]:
import os
import ast
import pandas as pd
from io import BytesIO
from io import StringIO
from dotenv import load_dotenv
from google.cloud import storage

# PARAMETERS

In [ ]:
load_dotenv("reloaded.env")
gcp_json = os.getenv('gcp_json')

In [ ]:
#_____CREAR CONEXIÓN CON GOOGLE CLOUD
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=gcp_json
storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_or_name="jnet")

In [ ]:
#_____IF I HAVE TXT DATAFRAME

payOutsDataFrame=bucket.get_blob(blob_name="payOutsDataFrame.txt")
payOutsDataFrame=payOutsDataFrame.download_as_string()
payOutsDataFrame=pd.DataFrame(ast.literal_eval(payOutsDataFrame.decode("utf-8")))

#_____IF I HAVE CSV DATAFRAME

payOutsDataFrame=bucket.get_blob(blob_name="payOutsDataFrame.csv")
columns_payOutsDataFrame=["id","amount","currency","status","amount_fee","currency_fee","eur_usdt_rate","amount (eur)","amount_fee (eur)","sourceWalletId","destination_type","destination_id","destination_name","trackingRef","createDate","multinet_amount"]
payOutsDataFrame=pd.read_csv(BytesIO(payOutsDataFrame), skiprows=1, names=columns_payOutsDataFrame, engine='python')

#_____UPLOAD FILE

blob=bucket.blob("payOutsDataFrame.txt")
blob.upload_from_string(data=str(payOutsDataFrame.to_dict()))

# FUNCTIONS

GENERALES

In [ ]:
# FUNCIÓN QUE REDONDEA FLOAT HACIA ABAJO SEGÚN DECIMALES
def round_decimals_down(number:float, decimals:int=2):
    """
    Returns a value rounded down to a specific number of decimal places.
    """
    if not isinstance(decimals, int):
        raise TypeError("decimal places must be an integer")
    elif decimals < 0:
        raise ValueError("decimal places has to be 0 or more")
    elif decimals == 0:
        return math.floor(number)

    factor = 10 ** decimals
    return math.floor(number * factor) / factorho,

#####

#FUNCIÓN QUE REDONDEA FLOAT HACIA ARRIBA SEGÚN DECIMALES
def round_decimals_up(number:float, decimals:int=2):
    """
    Returns a value rounded down to a specific number of decimal places.
    """
    if not isinstance(decimals, int):
        raise TypeError("decimal places must be an integer")
    elif decimals < 0:
        raise ValueError("decimal places has to be 0 or more")
    elif decimals == 0:
        return math.ceil(number)

    factor = 10 ** decimals
    return math.ceil(number * factor) / factor

0: ACTUALIZO BASE DE DATOS + BALANCEO + ACTUALIZAR FIAT + CRYPTO DISPONIBLES

In [ ]:
#ACTUALIA BALANCE CRYPTO
def getCRYinAccount():
    
    global CRYPT
    global client
    global sleep_api
    global balanceCRY
    global waitException
    
    while True:
        try:   
            balance = client.balance(CRYPT)
            balanceCRY = balance.amount.amount   
            break
        except: 
            time.sleep(sleep_api)
    
    return balance.amount.amount

#####

#ACTUALIA BALANCE FIAT
def getMONinAccount():
    
    global MONEY
    global client
    global sleep_api
    global balanceMON
    global waitException
    
    while True:
        try:
            balance = client.balance(MONEY)
            balanceMON = balance.amount.amount    
            break
        except:
            time.sleep(sleep_api)

    return balance.amount.amount

#####

#ACTUALIZA EL BALANCE DE ASKS Y BIDS EJECUTADOS
def updatePast_Asks_Bids():
    
    global MONEY
    global CRYPT
    global pastAsks
    global pastBids
    global database_past_asks_bids
    
    pastAsks=0.0
    pastBids=0.0
    pickle_in = open("database_past_asks_bids_"+MONEY+"_"+CRYPT+".pickle","rb")
    database_past_asks_bids= pickle.load(pickle_in)
    database_past_asks_bids.traded_amount=database_past_asks_bids.traded_amount.astype(float)
    database_past_asks_bids['created_at']= pd.to_datetime(database_past_asks_bids['created_at'])

    #_____

    pastAsks=database_past_asks_bids.loc[(database_past_asks_bids['type'] == "Ask") &
                                (database_past_asks_bids['price_type'] == "limit") &
                                (database_past_asks_bids["market_id"]==CRYPT+"-"+MONEY) &
                                ((database_past_asks_bids['state'] == "canceled")|
                                 (database_past_asks_bids['state'] == "traded")), 'traded_amount'].sum()

    pastBids=database_past_asks_bids.loc[(database_past_asks_bids['type'] == "Bid") &
                                (database_past_asks_bids['price_type'] == "limit") &
                                (database_past_asks_bids["market_id"]==CRYPT+"-"+MONEY) &
                                ((database_past_asks_bids['state'] == "canceled")|
                                 (database_past_asks_bids['state'] == "traded")), 'traded_amount'].sum()

    pastAsks=round(pastAsks,5)
    pastBids=round(pastBids,5)

#####
    
#ME DICE EL VOLUMEN QUE DEBO PONER
def history_trades():
    
    global MONEY
    global CRYPT
    global client
    global pastAsks
    global buyPrice
    global pastBids
    global sellPrice
    global minVolumeTrade
    
    updatePast_Asks_Bids()
    
    if abs(pastAsks-pastBids)<minVolumeTrade:
        
        sellPrice=0.0
        buyPrice=0.0
        
        #______
    
        myActualMoney=getMONinAccount()
        myActualCrypt=getCRYinAccount()
        recomendedVolume=myActualCrypt*0.8

        #_____SIMULAR MARKET ORDER CON LA RECOMENDED VOLUME QUE TENGO

        myBudaOrder=client.quotation_market(amount=myActualMoney,quotation_type="bid_given_size",market_id="btc-cop")
        myQuotation=myBudaOrder.quote_exchanged[0]

        #_____ 

        if recomendedVolume>myQuotation*0.8:
            recomendedVolume=myQuotation*0.8
                
    else:
        recomendedVolume=abs(pastAsks-pastBids)
    return round_decimals_down(recomendedVolume,5)

#_____

#ACTUALIZO BASE DE DATOS + BALANCEO + ACTUALIZAR FIAT + CRYPTO DISPONIBLES
def balancing_Ask_Bid():
    
    global pastAsks
    global pastBids
    global askVolume
    global bidVolume
    global askControl
    global bidControl
    global volume

    volume=history_trades()
    controlVariable=abs(pastAsks-pastBids)    
    
    if (pastAsks==pastBids) or (abs(pastAsks-pastBids)<minVolumeTrade):
        askVolume=volume
        bidVolume=volume
    elif pastAsks<pastBids:
        if (round_decimals_down(pastBids-pastAsks,5)>volume):
            subject = "SpreadNet: Stopped (Balancing Problem)"
            msg = "Revise SpreadNet"
            owners = json.loads(config.get('owner_information','OWNERS_ALERT'))
            enviar_alerta(subject, msg, owners)
        else:
            askVolume=round_decimals_down(pastBids-pastAsks,5) if (pastBids-pastAsks>=minVolumeTrade) else 0.0
            bidVolume=0.0
    elif pastAsks>pastBids:
        if (round_decimals_down(pastAsks-pastBids,5)>volume):
            subject = "SpreadNet: Stopped (Balancing Problem)"
            msg = "Revise SpreadNet"
            owners = json.loads(config.get('owner_information','OWNERS_ALERT'))
            enviar_alerta(subject, msg, owners)
            #printOutput(errorBalancingAskBid_2)
        else:
            bidVolume=round_decimals_down(pastAsks-pastBids,5) if (pastAsks-pastBids>=minVolumeTrade) else 0.0
            askVolume=0.0

1: ACTUALIZAR ORDENES EXISTENTES + CANCELO TODO SI TENGO ORDENES

In [ ]:
#ELIMINA TODAS LAS ORDENES EXISTENTES + ESCRIBO ODERNES EJECUTADAS
def finishThemAll():

    global client
    global bidOrderId
    global bidOrderDetails
    global last_volume_traded_fee
    global askOrderId
    global askOrderDetails
    global database_past_asks_bids
    global theorySellExecuted
    global theoryBuyExecuted
    global theorySellPrice
    global theoryBuyPrice
    global API_SECRET
    global API_KEY
    global MONEY
    global CRYPT
    global wait
    global waitException

    askPendings=0
    bidPendings=0
    
    client_surbtc = surbtc.Client(API_KEY,API_SECRET)
    Market = client_surbtc.getMarket(CRYPT+"-"+MONEY)
    data=pd.DataFrame(Market.getPendingOrders())
    
    if len(data)>0:
    
        askPendings=len(data.loc[((data.state=="pending") | (data.state=="accepted")) & (data.type=="Ask")])
        bidPendings=len(data.loc[((data.state=="pending") | (data.state=="accepted")) & (data.type=="Bid")])

        if (askPendings>=1) or (bidPendings>=1):

            askOrderId=None
            askOrderDetails=None
            askIdList=list(data.loc[((data.state=="pending") | (data.state=="accepted")) & (data.type=="Ask")].id.values)

            bidOrderId=None
            bidOrderDetails=None
            bidIdList=list(data.loc[((data.state=="pending") | (data.state=="accepted")) & (data.type=="Bid")].id.values)

            if len(askIdList)>=1:

                for i in askIdList:

                    time.sleep(round(wait/10))
                    while True:
                        try:
                            askOrderDetailsFinish = client.order_details(i)
                            break
                        except:
                            time.sleep(waitException)
                            #printOutput(f'client error waiting {waitException} seconds')
                    if (askOrderDetailsFinish.traded_amount.amount > 0.0):
                        theorySellExecuted=theorySellPrice
                        write_buy_sell_prices()
                    time.sleep(round(wait/10))
                    while True:
                        try:
                            client.cancel_order(i)
                            break
                        except:
                            time.sleep(waitException)
                            #printOutput(f'client error waiting {waitException} seconds')
                    while(True):
                        time.sleep(wait)
                        while True:
                            try:
                                askOrderDetailsFinish = client.order_details(i)
                                break
                            except:
                                time.sleep(waitException)
                                #printOutput(f'client error waiting {waitException} seconds')
                        if (askOrderDetailsFinish.state=="canceled") or (askOrderDetailsFinish.state=="traded"):
                            break
                        else:
                            time.sleep(wait/2)
                            while True:
                                try:
                                    client.cancel_order(i)
                                    break
                                except:
                                    time.sleep(waitException)
                                    #printOutput(f'client error waiting {waitException} seconds')

                    pickle_in = open("database_past_asks_bids_"+MONEY+"_"+CRYPT+".pickle","rb")
                    database_past_asks_bids= pickle.load(pickle_in)

                    columns=["id","account_id","amount","created_at","fee_currency","limit","market_id","original_amount","paid_fee","price_type","state","total_exchanged","traded_amount","type","json"]
                    appen_order_dataframe=pd.DataFrame(columns=columns)
                    order_dataframe=pd.DataFrame(askOrderDetailsFinish[14]).head(1)
                    appen_order_dataframe=appen_order_dataframe.append(order_dataframe,sort=False)
                    database_past_asks_bids=database_past_asks_bids.append(appen_order_dataframe,sort=False)
                    database_past_asks_bids.drop_duplicates(subset=["id"],inplace=True)
                    database_past_asks_bids.reset_index(drop=True,inplace=True)

                    pickle_out = open("database_past_asks_bids_"+MONEY+"_"+CRYPT+".pickle","wb")
                    pickle.dump(database_past_asks_bids, pickle_out)
                    pickle_out.close()

            if len(bidIdList)>=1:

                for j in bidIdList:

                    time.sleep(round(wait/10))
                    while True:
                        try:
                            bidOrderDetailsFinish = client.order_details(j)
                            break
                        except:
                            time.sleep(waitException)
                            #printOutput(f'client error waiting {waitException} seconds')
                    if (bidOrderDetailsFinish.traded_amount.amount > 0.0):
                        theoryBuyExecuted=theoryBuyPrice
                        last_volume_traded_fee=float(bidOrderDetailsFinish.paid_fee[0])+last_volume_traded_fee
                        createMarketBidFee()
                        write_buy_sell_prices()
                    time.sleep(round(wait/10))
                    while True:
                        try:
                            client.cancel_order(j)
                            break
                        except:
                            time.sleep(waitException)
                            #printOutput(f'client error waiting {waitException} seconds')
                    while(True):
                        time.sleep(wait)
                        while True:
                            try:
                                bidOrderDetailsFinish = client.order_details(j)
                                break
                            except:
                                time.sleep(waitException)
                                #printOutput(f'client error waiting {waitException} seconds')
                        if (bidOrderDetailsFinish.state=="canceled") or (bidOrderDetailsFinish.state=="traded"):
                            break
                        else:
                            time.sleep(wait/2)
                            while True:
                                try:
                                    client.cancel_order(j)
                                    break
                                except:
                                    time.sleep(waitException)
                                    #printOutput(f'client error waiting {waitException} seconds')

                    pickle_in = open("database_past_asks_bids_"+MONEY+"_"+CRYPT+".pickle","rb")
                    database_past_asks_bids= pickle.load(pickle_in)

                    columns=["id","account_id","amount","created_at","fee_currency","limit","market_id","original_amount","paid_fee","price_type","state","total_exchanged","traded_amount","type","json"]
                    appen_order_dataframe=pd.DataFrame(columns=columns)
                    order_dataframe=pd.DataFrame(bidOrderDetailsFinish[14]).head(1)
                    appen_order_dataframe=appen_order_dataframe.append(order_dataframe,sort=False)
                    database_past_asks_bids=database_past_asks_bids.append(appen_order_dataframe,sort=False)
                    database_past_asks_bids.drop_duplicates(subset=["id"],inplace=True)
                    database_past_asks_bids.reset_index(drop=True,inplace=True)

                    pickle_out = open("database_past_asks_bids_"+MONEY+"_"+CRYPT+".pickle","wb")
                    pickle.dump(database_past_asks_bids, pickle_out)
                    pickle_out.close()

        balancing_Ask_Bid()

2: ACTUALIZAR VARIABLE GLOBAL DE PRECIOS PUNTA DEL LIBRO DE ORDENES

In [ ]:
#ORDER BOOK
def request_order_book():
    global URL
    global API_KEY
    global API_SECRET
    global wait
    global waitException
    
    while True:
        try:
            with requests.get(URL, auth=BudaHMACAuth(API_KEY, API_SECRET)) as r:
                order_book = r.json()
                if ( order_book != None and 'order_book' in order_book ):
                    return order_book['order_book']
                    break
                #else:
                    #printOutput('Order book no llega completo')
        except:
            time.sleep(waitException)
            #printOutput('Order book no llega completo')

#ACTUALIZA PRECIOS PUNTA
def updateLimits():
    
    global limitAskPrice
    global limitBidPrice
    global limitAskVolume
    global limitBidVolume
    
    order_book = request_order_book()
    
    limitAskPrice = float(order_book['asks'][0][0])
    limitBidPrice = float(order_book['bids'][0][0])
    limitAskVolume = float(order_book['asks'][0][1])
    limitBidVolume = float(order_book['bids'][0][1])

3: COLOCA POSICION ASK + PRECIO ASK + DETALLE TRANSACCION ASK + ID TRANSACCION + TESTIGO DE QUE TENGO ORDEN ASK

In [ ]:
#CANCELAR ORDEN ASK
def cancelAsk():
    global client
    global askOrderId
    global askOrderDetails
    global database_past_asks_bids
    global theorySellExecuted
    global theorySellPrice
    global MONEY
    global CRYPT
    global wait
    global waitException
    
    time.sleep(round(wait/10))
    while True:
        try:
            askOrderDetails = client.order_details(askOrderId)
            break
        except:
            time.sleep(waitException)
            #printOutput(f'client error waiting {waitException} seconds')
    if (askOrderDetails.traded_amount.amount > 0.0):
        theorySellExecuted=theorySellPrice
        write_buy_sell_prices()
    time.sleep(round(wait/10))
    while True:
        try:
            client.cancel_order(askOrderId)
            break
        except:
            time.sleep(waitException)
            #printOutput(f'client error waiting {waitException} seconds')
    while(True):
        time.sleep(wait)
        while True:
            try:
                askOrderDetails = client.order_details(askOrderId)
                break
            except:
                time.sleep(waitException)
                #printOutput(f'client error waiting {waitException} seconds')
        if (askOrderDetails.state=="canceled") or (askOrderDetails.state=="traded"):
            break
        else:
            time.sleep(wait/2)
            while True:
                try:
                    client.cancel_order(askOrderId)
                    break
                except:
                    time.sleep(waitException)
                    #printOutput(f'client error waiting {waitException} seconds')
    
    pickle_in = open("database_past_asks_bids_"+MONEY+"_"+CRYPT+".pickle","rb")
    database_past_asks_bids= pickle.load(pickle_in)

    columns=["id","account_id","amount","created_at","fee_currency","limit","market_id","original_amount","paid_fee","price_type","state","total_exchanged","traded_amount","type","json"]
    appen_order_dataframe=pd.DataFrame(columns=columns)
    order_dataframe=pd.DataFrame(askOrderDetails[14]).head(1)
    appen_order_dataframe=appen_order_dataframe.append(order_dataframe,sort=False)
    database_past_asks_bids=database_past_asks_bids.append(appen_order_dataframe,sort=False)
    database_past_asks_bids.drop_duplicates(subset=["id"],inplace=True)
    database_past_asks_bids.reset_index(drop=True,inplace=True)

    pickle_out = open("database_past_asks_bids_"+MONEY+"_"+CRYPT+".pickle","wb")
    pickle.dump(database_past_asks_bids, pickle_out)
    pickle_out.close()
    
    balancing_Ask_Bid()

    #printOutput("Cancel Ask Order [ID]= ",askOrderId)
    askOrderDetails=None
    askOrderId=None

#CREAR POSICIÓN LIMIT ASK  A UN PRECIO DADO
def createAsk(limitAskPrice):
    
    global balanceCRY
    global areCRY
    global askOrderId
    global askOrderDetails
    global client
    global askVolume
    global bidVolume
    global pastAsks
    global pastBids
    global theorySellPrice
    global MONEY
    global CRYPT
    global wait
    global waitException
    
    while askOrderId!=None:
        try:
            cancelAsk()
        except:
            pass
            time.sleep(wait/2)
    getCRYinAccount()
    amountCRY = balanceCRY
    if round(askVolume,5) >= minVolumeTrade:
        #printOutput("[AskVolume, BidVolume]: =",askVolume,bidVolume)
        if (amountCRY >= askVolume):
            areCRY = True
            amountAsk = askVolume
            time.sleep(round(wait/10))
            while True:
                try:
                    orden = client.new_order(CRYPT.lower()+"-"+MONEY.lower(), "ask", "limit", amountAsk, limitAskPrice)
                    askOrderId = orden.id
                    askOrderDetails = client.order_details(askOrderId)
                    theorySellPrice=limitAskPrice
                    #printOutput("New Ask Order [Volume,Price,ID] = ",amountAsk,limitAskPrice,askOrderId)
                    break
                except:
                    time.sleep(waitException)
                    #printOutput(f'client error waiting {waitException} seconds')
        else:
            areCRY = False
            #printOutput("Stopped: No CRYPT --- createAsk(limitAskPrice)")
            #printOutput(errorCreateAsk)